# Using deep features to build an image classifier

# Fire up GraphLab Create

In [1]:
import graphlab

# Load a common image analysis dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set.  

In [2]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\anwar\AppData\Local\Temp\graphlab_server_1491663925.log.0
INFO:graphlab.cython.cy_server:GraphLab Create v2.1 started. Logging: C:\Users\anwar\AppData\Local\Temp\graphlab_server_1491663925.log.0


This non-commercial license of GraphLab Create for academic use is assigned to 1101071@eng.asu.edu.eg and will expire on April 10, 2017.


# Exploring the image data

In [3]:
graphlab.canvas.set_target('ipynb')

In [4]:
image_train['image'].show()

# Train a classifier on the raw image pixels

We first start by training a classifier on just the raw pixels of the image.

In [5]:
raw_pixel_model = graphlab.logistic_classifier.create(image_train,target='label',
                                              features=['image_array'], verbose=False)

# Make a prediction with the simple model based on raw pixels

In [6]:
image_test[0:3]['image'].show()

In [7]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [8]:
raw_pixel_model.predict(image_test[0:3])

dtype: str
Rows: 3
['bird', 'cat', 'bird']

The model makes wrong predictions for all three images.

# Evaluating raw pixel model on test data

In [9]:
raw_pixel_model.evaluate(image_test)

{'accuracy': 0.4735, 'auc': 0.7187816250000003, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     bird     |       dog       |  177  |
 |     dog      |       cat       |  239  |
 |     cat      |       cat       |  325  |
 |     bird     |    automobile   |  143  |
 |  automobile  |    automobile   |  633  |
 |     dog      |    automobile   |  109  |
 |     dog      |       dog       |  414  |
 |     cat      |       dog       |  288  |
 |     bird     |       cat       |  158  |
 |  automobile  |       bird      |  110  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.4700814605749145, 'log_loss': 1.2123878236304955, 'precision': 0.

The accuracy of this model is poor, getting only about 46% accuracy.

# Can we improve the model using deep features

We only have 2005 data points, so it is not possible to train a deep neural network effectively with so little data.  Instead, we will use transfer learning: using deep features trained on the full ImageNet dataset, we will train a simple model on this small dataset.

In [10]:
len(image_train)

2005

## Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [11]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

As we can see, the column deep_features already contains the pre-computed deep features for this data. 

In [12]:
image_train.head()

Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array

Rows: 10

Data:
+-----+----------------------+------------+-------------------------------+
|  id |        image         |   label    |         deep_features         |
+-----+----------------------+------------+-------------------------------+
|  24 | Height: 32 Width: 32 |    bird    | [0.242871761322, 1.0954537... |
|  33 | Height: 32 Width: 32 |    cat     | [0.525087952614, 0.0, 0.0,... |
|  36 | Height: 32 Width: 32 |    cat     | [0.566015958786, 0.0, 0.0,... |
|  70 | Height: 32 Width: 32 |    dog     | [1.12979578972, 0.0, 0.0, ... |
|  90 | Height: 32 Width: 32 |    bird    | [1.71786928177, 0.0, 0.0, ... |
|  97 | Height: 32 Width: 32 | automobile | [1.57818555832, 0.0, 0.0, ... |
| 107 | Height: 32 Width: 32 |    dog     | [0.0, 0.0, 0.220677852631,... |
| 121 | Height: 32 Width: 32 |    bird    | [0.0, 0.23753464222, 0.0, ... |
| 136 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
| 138 | Height: 32 Width: 32 |    bird    | [0.658935725689, 0.0, 0.0,... |
+-----+----------------------+------------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [73.0, 77.0, 58.0, 71.0, 6... |
| [7.0, 5.0, 8.0, 7.0, 5.0, ... |
| [169.0, 122.0, 65.0, 131.0... |
| [154.0, 179.0, 152.0, 159.... |
| [216.0, 195.0, 180.0, 201.... |
| [33.0, 44.0, 27.0, 29.0, 4... |
| [97.0, 51.0, 31.0, 104.0, ... |
| [93.0, 96.0, 88.0, 102.0, ... |
| [35.0, 59.0, 53.0, 36.0, 5... |
| [205.0, 193.0, 195.0, 200.... |
+-------------------------------+
[10 rows x 5 columns]

# Given the deep features, let's train a classifier

In [13]:
deep_features_model = graphlab.logistic_classifier.create(image_train,
                                                         features=['deep_features'],
                                                         target='label', verbose=False)

# Apply the deep features model to first few images of test set

In [14]:
image_test[0:3]['image'].show()

In [15]:
deep_features_model.predict(image_test[0:3])

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

The classifier with deep features gets all of these images right!

# Compute test_data accuracy of deep_features_model

As we can see, deep features provide us with significantly better accuracy (about 78%)

In [16]:
deep_features_model.evaluate(image_test)

{'accuracy': 0.783, 'auc': 0.9404363749999987, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     bird     |       cat       |  153  |
 |     dog      |       dog       |  733  |
 |     cat      |       dog       |  220  |
 |  automobile  |       dog       |   5   |
 |     cat      |    automobile   |   31  |
 |     dog      |       bird      |   37  |
 |     cat      |       bird      |   60  |
 |     bird     |       dog       |   60  |
 |  automobile  |    automobile   |  950  |
 |     bird     |    automobile   |   27  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.7841752489122076, 'log_loss': 0.5933408913283521, 'precision': 0.7

As you can see, the accuracy has raised to 78%

# Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [17]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

# Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [18]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [19]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 101.094ms    |

| Done         |         | 100         | 491.351ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |       384       |      0.0      |  1   |
|      0      |       6910      | 36.9403137951 |  2   |
|      0      |      39777      | 38.4634888975 |  3   |
|      0      |      36870      | 39.7559623119 |  4   |
|      0      |      41734      | 39.7866014148 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

We are going to create a simple function to view the nearest neighbors to save typing:

In [20]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [21]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 23.013ms     |

| Done         |         | 100         | 320.211ms    |

+--------------+---------+-------------+--------------+

In [22]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

## Finding similar images to a car

In [23]:
car = image_train[8:9]
car['image'].show()

In [24]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 22.017ms     |

| Done         |         | 100         | 367.243ms    |

+--------------+---------+-------------+--------------+

# Just for fun, let's create a lambda to find and show nearest neighbor images

In [25]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [26]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 18.014ms     |

| Done         |         | 100         | 382.254ms    |

+--------------+---------+-------------+--------------+

In [27]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 21.016ms     |

| Done         |         | 100         | 401.267ms    |

+--------------+---------+-------------+--------------+

In [28]:
image_train["label"].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


**Quiz Answer(1)**

bird

## Creating category-specific image retrieval models: 
In most retrieval tasks, the data we have is unlabeled, thus we call these unsupervised learning problems. However, we have labels in this image dataset, and will use these to create one model for each of the 4 image categories, {‘dog’,’cat’,’automobile’,bird’}. To start, follow these steps:

- Split the SFrame with the training data into 4 different SFrames. Each of these will contain data for 1 of the 4 categories above. Hint: if you use a logical filter to select the rows where the ‘label’ column equals ‘dog’, you can create an SFrame with only the data for images labeled ‘dog’.

- Similarly to the image retrieval part, you are going to create a nearest neighbor model using the 'deep_features' as the features, but this time create one such model for each category, using the corresponding subset of the training_data. You can call the model with the ‘dog’ data the dog_model, the one with the ‘cat’ data the cat_model, as so on.

You now have a nearest neighbors model that can find the nearest ‘dog’ to any image you give it, the dog_model; one that can find the nearest ‘cat’, the cat_model; and so on.

In [29]:
dog_image_train = image_train.filter_by('dog', 'label')
cat_image_train = image_train.filter_by('cat', 'label')
automobile_image_train = image_train.filter_by('automobile', 'label')
bird_image_train = image_train.filter_by('bird', 'label')

In [30]:
dog_image_train.head()

Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array

Rows: 10

Data:
+------+----------------------+-------+-------------------------------+
|  id  |        image         | label |         deep_features         |
+------+----------------------+-------+-------------------------------+
|  70  | Height: 32 Width: 32 |  dog  | [1.12979578972, 0.0, 0.0, ... |
| 107  | Height: 32 Width: 32 |  dog  | [0.0, 0.0, 0.220677852631,... |
| 177  | Height: 32 Width: 32 |  dog  | [0.0, 1.45965671539, 0.0, ... |
| 424  | Height: 32 Width: 32 |  dog  | [0.942399680614, 0.0, 0.22... |
| 462  | Height: 32 Width: 32 |  dog  | [1.43462562561, 0.0, 0.0, ... |
| 542  | Height: 32 Width: 32 |  dog  | [0.451547086239, 0.0, 0.0,... |
| 573  | Height: 32 Width: 32 |  dog  | [0.592360973358, 0.0, 0.0,... |
| 851  | Height: 32 Width: 32 |  dog  | [0.690123438835, 0.0, 0.0,... |
| 919  | Height: 32 Width: 32 |  dog  | [0.0, 0.177558660507, 0.13... |
| 1172 | Height: 32 Width: 32 |  dog  | [0.517601490021, 0.0, 1.96... |
+------+----------------------+-------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [154.0, 179.0, 152.0, 159.... |
| [97.0, 51.0, 31.0, 104.0, ... |
| [55.0, 75.0, 42.0, 51.0, 7... |
| [60.0, 35.0, 18.0, 63.0, 4... |
| [86.0, 69.0, 75.0, 57.0, 4... |
| [196.0, 174.0, 113.0, 140.... |
| [153.0, 103.0, 52.0, 151.0... |
| [39.0, 6.0, 4.0, 53.0, 23.... |
| [29.0, 43.0, 4.0, 24.0, 35... |
| [182.0, 180.0, 197.0, 196.... |
+-------------------------------+
[10 rows x 5 columns]

In [31]:
dog_model = graphlab.nearest_neighbors.create(dog_image_train, features=['deep_features'], label='id')
cat_model = graphlab.nearest_neighbors.create(cat_image_train, features=['deep_features'], label='id')
automobile_model = graphlab.nearest_neighbors.create(automobile_image_train, features=['deep_features'], label='id')
bird_model = graphlab.nearest_neighbors.create(bird_image_train, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

In [32]:
def show_nearest_neighbors(i, knn_model):
    return get_images_from_ids(knn_model.query(image_test[i:i+1]))['image'].show()

show_nearest_neighbors(0, cat_model)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 18.016ms     |

| Done         |         | 100         | 111.074ms    |

+--------------+---------+-------------+--------------+

**Quiz Answer (2)**

[last image]

In [33]:
show_nearest_neighbors(0, dog_model)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 20.013ms     |

| Done         |         | 100         | 138.09ms     |

+--------------+---------+-------------+--------------+

**Quiz Answer (3)**

[4]

In [34]:
cat_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 16.018ms     |

| Done         |         | 100         | 100.066ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16289      |  34.623719208 |  1   |
|      0      |      45646      | 36.0068799284 |  2   |
|      0      |      32139      | 36.5200813436 |  3   |
|      0      |      25713      | 36.7548502521 |  4   |
|      0      |       331       | 36.8731228168 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [35]:
cat_model.query(image_test[0:1])["distance"].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 57.042ms     |

| Done         |         | 100         | 171.113ms    |

+--------------+---------+-------------+--------------+

36.15573070978294

**Quiz Answer(4)**

35 to 37

In [36]:
dog_model.query(image_test[0:1])["distance"].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 20.013ms     |

| Done         |         | 100         | 127.084ms    |

+--------------+---------+-------------+--------------+

37.77071136184156

**Quiz Answer(5)**

37 to 39

**Quiz Answer(6)**

cat

### Finding nearest neighbors in the training set for each part of the test set:
Thus far, we have queried our nearest neighbors models with a single image as the input, but you can actually query with a whole set of data, and it will find the nearest neighbors for each data point. Note that the input index will be stored in the ‘query_label’ column of the output SFrame.

Create an SFrame with the distances from ‘dog’ test examples to the respective nearest neighbors in each class in the training data: The ‘distance’ column in dog_cat_neighbors contains the distance between each ‘dog’ image in the test set and its nearest ‘cat’ image in the training set. The question we want to answer is how many of the test set ‘dog’ images are closer to a ‘dog’ in the training set than to a ‘cat’, ‘automobile’ or ‘bird’. So, next we will create an SFrame containing just these distances per data point. The goal is to create an SFrame called dog_distances with 4 columns:
- dog_distances[‘dog-dog’] ---- storing dog_dog_neighbors[‘distance’]
- dog_distances[‘dog-cat’] ---- storing cat_dog_neighbors[‘distance’]
- dog_distances[‘dog-automobile’] ---- storing automobile_dog_neighbors[‘distance’]
- dog_distances[‘dog-bird’] ---- storing bird_dog_neighbors[‘distance’]

In [37]:
dog_image_test = image_test.filter_by('dog', 'label')

In [38]:
dog_dog_neighbors = dog_model.query(dog_image_test, k=1)
cat_dog_neighbors = cat_model.query(dog_image_test, k=1)
automobile_dog_neighbors = automobile_model.query(dog_image_test, k=1)
bird_dog_neighbors = bird_model.query(dog_image_test, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 120000  | 25.1046     | 946.625ms    |

| Done         | 478000  | 100         | 1.02s        |

+--------------+---------+-------------+--------------+

In [39]:
dog_distances = graphlab.SFrame({   'dog_dog': dog_dog_neighbors['distance'],
                                    'cat_dog': cat_dog_neighbors['distance'],
                                    'automobile_dog': automobile_dog_neighbors['distance'],
                                    'bird_dog': bird_dog_neighbors['distance']
                                })
dog_distances.head()

Columns:
	automobile_dog	float
	bird_dog	float
	cat_dog	float
	dog_dog	float

Rows: 10

Data:
+----------------+---------------+---------------+---------------+
| automobile_dog |    bird_dog   |    cat_dog    |    dog_dog    |
+----------------+---------------+---------------+---------------+
| 41.9579761457  | 41.7538647304 | 36.4196077068 | 33.4773590373 |
| 46.0021331807  | 41.3382958925 | 38.8353268874 | 32.8458495684 |
| 42.9462290692  | 38.6157590853 | 36.9763410854 | 35.0397073189 |
| 41.6866060048  | 37.0892269954 | 34.5750072914 | 33.9010327697 |
| 39.2269664935  |  38.272288694 |  34.778824791 | 37.4849250909 |
| 40.5845117698  | 39.1462089236 | 35.1171578292 |  34.945165344 |
| 45.1067352961  |  40.523040106 | 40.6095830913 | 39.0957278345 |
| 41.3221140974  | 38.1947918393 | 39.9036867306 | 37.7696131032 |
| 41.8244654995  | 40.1567131661 | 38.0674700168 | 35.1089144603 |
| 45.4976929401  | 45.5597962603 | 42.7258732951 | 43.2422832585 |
+----------------+---------------+---------------+---------------+
[10 rows x 4 columns]

### Computing the number of correct predictions using 1-nearest neighbors for the dog class: 
Now that you have created the SFrame dog_distances, you will  use the method **apply()** on this SFrame to iterate line by line and compute the number of ‘dog’ test examples where the distance to the nearest ‘dog’ was lower than that to the other classes. You will do this in three steps:

- Consider one row of the SFrame dog_distances. Let’s call this variable **row**. Create a function starting with **def is_dog_correct(row):** which returns 1 if the value for row[‘dog-dog’] is lower than that of the other columns, and 0 otherwise. That is, returns 1 if this row is correctly classified by 1-nearest neighbors, and 0 otherwise.

- Using the function is_dog_correct(row), you can check if 1 row is correctly classified. Now, you want to count how many rows are correctly classified. You could do a for loop iterating through each row and applying the function is_dog_correct(row). This method will be really slow, because the SFrame is not optimized for this type of operation. Instead, we will use the .apply() method to iterate the function is_dog_correct for each row of the SFrame.

- Computing the number of correct predictions for ‘dog’. You can now call **dog_distances.apply(is_dog_correct)** which will return an SArray (a column of data) with a 1 for every correct row and a 0 for every incorrect one.


In [50]:
row = dog_distances[0]

def is_dog_correct(row):
    if (row["dog_dog"] <= row["cat_dog"]) and (row["dog_dog"] <= row["bird_dog"]) and (row["dog_dog"] < row["automobile_dog"]):
        return 1
    else:
        return 0

is_dog_correct(row)

1

In [56]:
#it would take sometime
correctness = dog_distances.apply(is_dog_correct)

In [58]:
correctness.mean()

0.678

**Quiz Answer(7)**

60 to 70